In [1]:

##import packages
from tensorflow.keras import preprocessing, utils
from keras.backend import clear_session
import os
from gensim.models import Word2Vec
import re
import numpy as np
import yaml
import tensorflow as tf


2022-12-27 14:53:40.189413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-27 14:53:41.175695: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-27 14:53:41.175767: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-27 14:53:41.317383: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-27 14:53:44.756689: W tensorflow/stream_executor/platform/de

In [2]:
##loading the path
dir_path = './data/'
files_list = os.listdir(dir_path + os.sep)
     

In [3]:
#question list
questions = list()
answers = list() #answers list from yml file

for filepath in files_list:#opens all yml file, read it, take the questions and answers from it. 
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)##load yml file
    conversations = docs['conversations'] ##take conversations key part
    for con in conversations:
        if len( con ) > 2 : ##if the length of con means under the conversation part all the text's len if >2 
            questions.append(con[0]) ##the frist part will be added
            replies = con[ 1 : ] ##after the first part added, the replies are added.
            ans = '' ##ans has empty list
            for rep in replies: ##take rep from replies list
                ans += ' ' + rep ##adding all text and make sentenses
            answers.append( ans ) ##append it in answers list
        elif len( con )> 1: ##if the con has more than 1 length
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list() ##tags appendings
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( ' ' + answers_with_tags[i] + ' ' )

tokenizer = preprocessing.text.Tokenizer()##convert text to sequence of integers 
tokenizer.fit_on_texts( questions + answers )##fit it 
VOCAB_SIZE = len( tokenizer.word_index )+1 ##check the sequence of integers length
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 418


In [4]:


vocab = []
for word in tokenizer.word_index: ##then take the integer index of word which mostly common and maximum
  vocab.append(word)##append it in vocab

def tokenize(sentences):##tokenize function
  tokens_list = []##tokens_list
  vocabulary = []##vocabulary_list
  for sentence in sentences:
    sentence = sentence.lower()##convert each of the letter into lower
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)##replaced specific pattern to the sentence 
    tokens = sentence.split() ##split the words from sentence
    vocabulary += tokens##added the vocabulary
    tokens_list.append(tokens)##append it into the tokens_list
  return tokens_list, vocabulary
     


In [5]:
tokenized_questions = tokenizer.texts_to_sequences( questions )##again convert into integer sequences
maxlen_questions = max( [len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post')
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape, maxlen_questions)
     


# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )
     


(111, 9) 9
(111, 26) 26


In [6]:
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )##convert into 2d Numpy array
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )##convert class vectors or integer into binary matrix
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

(111, 26, 418)


In [7]:

encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)##nlp related work
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer="Adam", loss='categorical_crossentropy')

model.summary()

2022-12-27 14:54:24.806082: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-27 14:54:24.806823: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-27 14:54:24.806950: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hrithik-Lenovo-ideapad-320-15ISK): /proc/driver/nvidia/version does not exist
2022-12-27 14:54:24.808532: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 9)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 26)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 9, 200)       83600       ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 26, 200)      83600       ['input_2[0][0]']                
                                                                                              

In [8]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=5, epochs=200 ,verbose=1) 
model.save( 'model.h5' )

Epoch 1/200
23/23 [==============================] - 13s 78ms/step - loss: 2.0721
Epoch 2/200
23/23 [==============================] - 2s 78ms/step - loss: 1.8728
Epoch 3/200
23/23 [==============================] - 2s 83ms/step - loss: 1.7976
Epoch 4/200
23/23 [==============================] - 2s 78ms/step - loss: 1.7631
Epoch 5/200
23/23 [==============================] - 2s 82ms/step - loss: 1.7205
Epoch 6/200
23/23 [==============================] - 2s 80ms/step - loss: 1.6704
Epoch 7/200
23/23 [==============================] - 2s 78ms/step - loss: 1.6097
Epoch 8/200
23/23 [==============================] - 2s 84ms/step - loss: 1.5442
Epoch 9/200
23/23 [==============================] - 2s 79ms/step - loss: 1.4745
Epoch 10/200
23/23 [==============================] - 2s 82ms/step - loss: 1.4030
Epoch 11/200
23/23 [==============================] - 2s 80ms/step - loss: 1.3305
Epoch 12/200
23/23 [==============================] - 2s 78ms/step - loss: 1.2553
Epoch 13/200
23/23 [====

In [9]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [10]:

def str_to_tokens( sentence : str ):

    words = sentence.lower().split()
    tokens_list = list()
  
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [11]:
from keras.models import load_model

In [12]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    #empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

1/1 [==============================] - 0s 41ms/step
 software is the field of science which concerns itself with building hardware and software that replicates the functions of the human mind software software software that is
1/1 [==============================] - 0s 77ms/step
 software work on all operating systems including windows linux and mac os software software software will you software me software software that there is software that i
1/1 [==============================] - 0s 74ms/step
 software do that alwasy smiling yes i am there tell me software software software software that there is a resemblance between us software software software that on
1/1 [==============================] - 0s 67ms/step
 software runs on all operating systems linux and mac os software software software will you i am listening software software software that on all there is software
1/1 [==============================] - 0s 42ms/step


KeyboardInterrupt: 